Preprocess data

In [1]:
import pandas as pd
from tqdm import tqdm
import csv

In [2]:
brandbankExtract = r'C:\Users\admin_1878\Documents\Brandbank Data\Extracted Data\Brandbank Extract.csv'

In [ ]:
df = pd.read_csv(brandbankExtract)
df=df.fillna('')
df['text'] = df[[
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves']].agg(' '.join, axis=1)
df=df.drop(['name'
    ,'pl2','pl3',
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves'],axis=1)
df.rename({'pl1':'category'},axis=1, inplace=True)
df.drop(df[df['category'] =='Kruidenierswaren'].index, inplace=True)#onyl 1 example of this so dropping for simplicity
df['category'] = df['category'].str.replace(" ", "_")
df = df[df['category'] != '']
df = df[df['category'] != 'Unallocated']
df['text'] = df['text'].str.lower()
df.head()

RAKE_NLTK

In [ ]:
from rake_nltk import Rake
import nltk

In [ ]:
#nltk.download('stopwords') #Only needs to be run once to download stop words

In [ ]:
def get_num_words(row):
    return len(str(row['Phrase']).split())

def get_perc_phrase(row,total):
    return (float(row['Score'])/total*100)

In [ ]:
def get_Rake_phrases(df):
    r = Rake()
    r.extract_keywords_from_sentences(df['text'])
    phrases=r.get_ranked_phrases_with_scores()
    df = pd.DataFrame(phrases, columns = ['Rank', 'Phrase'])
    df['Phrase'] = df['Phrase'].str.strip()
    phrase_counts = df['Phrase'].value_counts()
    phrase_counts = phrase_counts.reset_index()
    phrase_counts.columns=['Phrase','Score']
    phrase_counts['Length'] = phrase_counts.apply (lambda row: get_num_words(row), axis=1)
    phrase_counts=phrase_counts.loc[phrase_counts['Length'] <=6]
    total = float(phrase_counts['Score'].sum())
    phrase_counts['Perc'] = phrase_counts.apply (lambda row: get_perc_phrase(row,total), axis=1)
    return phrase_counts

In [ ]:
df_raked = get_Rake_phrases(df)
#phrase_counts.to_csv(r'C:\Users\Clamfighter\Documents\GitHub\NLP\Impact Score Key Phrase\Extracted Phrases\Rake NLTK\Brandbank Latest Phrase Extraction.csv',index = False)

In [ ]:
phrase_count=[]

df_phraseCount = df_raked.loc[df_raked['Score'] != 1] #filters Score of 1 out. will be added back in later
phrases = df_phraseCount['Phrase']
text = df['text']
for phrase in phrases: #find number of phrases for all strings
    phrase_count.append(sum(phrase in s for s in text))
    print (phrase)
phrase_count

In [ ]:
df_phraseCount['Count'] = phrase_count
df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)
df_out = df_raked.merge(df_phraseCount, on='Phrase', how='left')
df_out.fillna(1,inplace=True)
df_out

In [ ]:
df_out.to_csv(r'out_lower.csv',index = False)